In [ ]:
%%capture
%load_ext autoreload
%autoreload 2

%pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np

In [5]:
flights = pd.read_parquet("../resources/flights.parquet")
flights.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0


In [7]:
# Registros nulos para cada campo
flights.isna().sum()

Flight_ID                0
Aeronave             80390
DepartureStation         1
ArrivalStation           1
Destination_Type         1
Origin_Type              1
STD                      0
STA                      0
Capacity                 0
Passengers          123525
Bookings            123525
dtype: int64

Se eliminan todos los casos donde `Aeropuerto` es nulo pues indica que el vuelo fue cancelado o cualquier otra situación que no es necesario considerar. Asimismo se eliminan 2 registro único donde los campos `DepartureStation`, `ArrivalStation`, `Destination_Type`, `Origin_Type` son nulos.

In [15]:
flights_ = flights.dropna(subset=["Aeronave", "Passengers", "Bookings"])
flights_ = flights_.dropna(subset=["DepartureStation", "ArrivalStation"])
flights_.isna().sum()

Flight_ID           0
Aeronave            0
DepartureStation    0
ArrivalStation      0
Destination_Type    0
Origin_Type         0
STD                 0
STA                 0
Capacity            0
Passengers          0
Bookings            0
dtype: int64

Se crea una columna de `Semana` y `Mes` para análisis posteriores.

In [32]:
flights_["STD"] = pd.to_datetime(flights_["STD"])
flights_["STA"] = pd.to_datetime(flights_["STA"])
flights_["Semana"] = flights_["STD"].dt.isocalendar().week
flights_["Mes"] = flights_["STD"].dt.month
flights_

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,Week,Month,Semana,Mes
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,42,10,42,10
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,27,7,27,7
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,26,6,26,6
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,6,2,6,2
5,3b5df8805161ea827d2f2e4298c38e06,XA-VBY,AJ,AR,Playa,MX Amigos y Familia,2023-09-07 17:10:00,2023-09-07 18:05:00,240,183.0,125.0,36,9,36,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245718,d9ef769565064b1ff1ad90d6d18f69cb,XA-VYF,AO,BT,Playa,Ciudad Principal,2023-12-26 06:20:00,2023-12-26 10:10:00,180,166.0,89.0,52,12,52,12
245745,598035669a4f0fa2ce1207977932df9e,XA-VAI,AW,BT,Playa,Ciudad Principal,2023-12-28 10:25:00,2023-12-28 13:50:00,180,142.0,54.0,52,12,52,12
245747,90d2cab80b7700b5d854ae393678144b,XA-VYA,BA,BT,Playa,Ciudad Principal,2023-12-23 11:45:00,2023-12-23 15:00:00,180,156.0,61.0,51,12,51,12
245748,cc7c1c5e6fd132fd0bdab3a35aac33c0,XA-VBK,BM,BT,Playa,Ciudad Fronteriza,2023-12-29 07:30:00,2023-12-29 14:50:00,240,189.0,86.0,52,12,52,12


In [33]:
flights_.to_parquet("../resources/flights_clean.parquet", index=False)